In [ ]:
import gsd
import gsd.hoomd
import numpy
from matplotlib import pyplot
%matplotlib inline

In [ ]:
import gsd
import gsd.hoomd
t = gsd.hoomd.open('trajectory.gsd', 'rb');
last_frame = t[-1]
box = [last_frame.configuration.box[0], last_frame.configuration.box[1]]

In [ ]:
import freud

In [ ]:
hex = freud.order.HexOrderParameter(rmax=1.2, k=6)

In [ ]:
psi = []
for frame in t[-10:]:
    hex.compute(box, frame.particles.position)
    print(numpy.abs(numpy.mean(hex.psi)))
    psi.append(numpy.mean(hex.psi))
print()
print(numpy.abs(numpy.mean(psi)))

In [ ]:
poly_area = 0.594410322684471  # TODO, get from job document
N = 440 # TODO: get from job document
# alternately, use a custom log operation to log phi directly

data = numpy.genfromtxt(fname='log.dat', skip_header=True)
pyplot.figure(figsize=(4,2.2), dpi=140);
pyplot.plot(data[:,0], N*poly_area/data[:,1]);
pyplot.ylabel('packing fraction');
pyplot.xlabel('time step');